In [ ]:

#1. Define the problem 

util = (c,cond) -> cond ? log(c) : -Inf #I wonder to what extend we can change this to the true definition we have in the slides; this is a simplifation.   
R = (k, ls, α, δ)->  ls* k^α + (1.0-δ)*k

#we define the R because we will optimize wrt k' and we will introduce k' in the loop implicitly.


# 2) Define some parameters
α    = 0.33    # Capital elasticity in production function
β    = 0.95    # Discount factor
δ    = 0.06    # Depreciation rate
σ    = 0.2     # elasticity of
param= [α,β,δ,σ]


#transition states

include("Markov_Chain_tools.jl")

ρ    = 0.8     # persistence of the shock
σ    = 0.05    # Volatility of the innovation
μ    = 0       # Average of the process
N    = 5       # Number of points in the grid of shocks
Δ    = 3.0     # number of std deviations away from the mean (Tauchen only)
la_t,P_t = tauchen(N,ρ,σ,μ,Δ)   # If Tauchen's (1986) method is chosen
la_rou,P_rou = rouwenhorst(N,ρ,σ,μ) # If Rouwenhorst's (1995) method is chosen


ls= exp.(la_t)
Ns= 5  ##number of states we are using.
#in the next case I will do it manually, but I understand the idea completely. 

#define grid of values of Kt, our state variable

kss  = (α*β/(1.0-β*(1.0-δ)))^(1.0/(1.0-α)) # Deterministic steady state of capital
Δk   = 0.9
kmin = (1.0-Δk)*kss
kmax = (1.0+Δk)*kss

Nk= 1000

kgrid= range(kmin, kmax, Nk)

#now we define a vector of resources per k, s of the grid. This is the potential resources we may have in the future.

Ri= [R(kgrid[ik],ls[is],α, δ ) for ik in 1:Nk, is in 1:Ns]


1000×5 Matrix{Float64}:
  1.0862    1.16855   1.26188   1.36763   1.48746
  1.09828   1.18112   1.275     1.38137   1.50191
  1.11032   1.19364   1.28806   1.39506   1.51629
  1.12231   1.20612   1.30107   1.40868   1.53061
  1.13427   1.21854   1.31403   1.42224   1.54485
  1.14619   1.23092   1.32694   1.43574   1.55904
  1.15807   1.24326   1.3398    1.44919   1.57315
  1.16991   1.25556   1.35261   1.46259   1.58721
  1.18171   1.26781   1.36538   1.47593   1.60121
  1.19348   1.28003   1.37809   1.48922   1.61514
  ⋮                                      
 10.4483   10.6654   10.9114   11.1901   11.5059
 10.4573   10.6744   10.9204   11.1993   11.5152
 10.4662   10.6834   10.9295   11.2084   11.5245
 10.4751   10.6924   10.9386   11.2176   11.5337
 10.4841   10.7014   10.9477   11.2268   11.543
 10.493    10.7104   10.9568   11.2359   11.5523
 10.5019   10.7194   10.9659   11.2451   11.5616
 10.5109   10.7284   10.9749   11.2543   11.5708
 10.5198   10.7374   10.984    11.2635   11

In [8]:


using PyPlot, Interpolations

function expectation(x, P)
    x*P'
end 


function t_operator(v, kgrid, param, P, Ri) #This is just a function to find the T operator (opimize right hand side)

α,β,δ,σ = param


Tv= similar(v)
Gk= similar(v)
Nk, Ns= size(v)
Ev= expectation(v, P)

    for is in 1:Ns
        index=1
            for ik in 1:Nk ##loop over 1000
                Vmax=-Inf
                k_prime=0.0
                R= Ri[ik, is]

                for ij in index:Nk
                    kp= kgrid[ij]
                    cond= (R- kp>0) & (kp-(1-δ)*kgrid[ik]>0)
                    vtmp= util(R- kp, cond)+ β*Ev[ij, is] ###or ij?
                    if (vtmp>=Vmax)
                        kp= kgrid[ij]
                        index= ij ##use the fact that policy is strictly increasing in capital
                        k_prime= kp
                        Vmax= vtmp
                    else 
                        break
                    end 
                end 

                Tv[ik, is]= Vmax
                Gk[ik, is]= k_prime
            end
    end 
 return Tv, Gk
end 

t_operator (generic function with 1 method)

In [9]:
function bellman_loop(kgrid, Ns, param, P, Ri;
                      tolerance = 1e-6,
                      criterion = :value)

    Nk = length(kgrid)
    v   = zeros(Float64, Nk, Ns)
    Gk0 = similar(v)

    err = Inf
    Tv = similar(v)
    Gk = similar(v)

    while err > tolerance
        Tv, Gk = t_operator(v, kgrid, param, P, Ri)

        if criterion == :value
            err = maximum(abs.(Tv .- v))
        elseif criterion == :policy
            err = maximum(abs.(Gk .- Gk0))
        else
            error("criterion must be :value or :policy")
        end

        v .= Tv      # in-place update
        Gk0 .= Gk
    end

    return v, Gk
end



bellman_loop (generic function with 1 method)

In [10]:
@time v, Gk= bellman_loop(kgrid, Ns, param, P_rou, Ri;
                      tolerance = 1e-6,
                      criterion = :value)



  0.662640 seconds (19.36 M allocations: 501.500 MiB, 5.46% gc time, 71.11% compilation time)


([-0.718594100535768 0.09223742730172546 … 1.7682823952573135 2.6332057852215796; -0.6865804343772883 0.12310346233352243 … 1.7970362234698054 2.6610399935800237; … ; 8.39707042415605 8.938712940931051 … 10.054685019174446 10.646480263906234; 8.40144233545361 8.94312871510838 … 10.058976685300676 10.650653044261075], [0.712641014188646 0.7664252416745816 … 0.8919217724750979 0.9725981137040012; 0.7216050521029685 0.7753892795889041 … 0.9008858103894205 0.9815621516183238; … ; 8.878879554136528 8.878879554136528 … 9.120908577823238 9.353973563595625; 8.887843592050851 8.887843592050851 … 9.129872615737561 9.353973563595625])

In [11]:
# Finding Steady State
ks=[]
for i in 1:N
    G=LinearInterpolation(kgrid,Gk[:,i],extrapolation_bc=Line()) #make it a continuous function. k=g(k,s), for each s we have we are finding the fixed point.
    crit=1
    tol=1e-6
    k0=kss
    while crit>tol
        k1=G(k0)
        crit=abs(k1-k0)
        k0=copy(k1)
    end
    push!(ks,k0)
end
ks

5-element Vector{Any}:
 2.8998662652833587
 3.8141981325442633
 4.975041042449039
 6.503409506841043
 8.529282075477948

In [12]:
fst=14
figure(figsize=(12,8))
Gc=[R(kgrid[ik],exp(ls[ia]),α,δ).-Gk[ik,ia] for ik in 1:Nk,ia in 1:Ns]
bg=[0.95,0.95,0.95]
ax=subplot(3,1,1,facecolor=bg)
for sp in ("bottom","top","left","right")
      ax.spines[sp].set_color("none")
      ax.tick_params(axis="y", width=0, length=0)
      ax.tick_params(axis="x", width=0, length=0)        
  end
ax.grid(color="w",linestyle="-",alpha=0.5,linewidth=1.5,zorder=0)
ax.plot(kgrid,v,color="k",linewidth=1)
xlim(kgrid[1],kgrid[Nk])
xlabel(L"k_t",fontsize=12)                                         # put a label on x-axis the L forces LaTeX code
title("Value Function",fontsize=fst)

ax=subplot(3,3,4,facecolor=bg)
for sp in ("bottom","top","left","right")
      ax.spines[sp].set_color("none")
      ax.tick_params(axis="y", width=0, length=0)
      ax.tick_params(axis="x", width=0, length=0)        
  end
ax.grid(color="w",linestyle="-",alpha=0.5,linewidth=1.5,zorder=0)
ax.plot(kgrid,Gk,color="k",linewidth=1)                             # Plots k(t+1)=G(k(t))
ax.plot(kgrid,kgrid,color="k",linewidth=1,linestyle="--") # Plots the 45° line
xlabel(L"k_t",fontsize=12)                                         # put a label on x-axis the L forces LaTeX code
ylabel(L"k_{t+1}=G_k(k_t)",fontsize=12)                              # put a label on y-axis the L forces LaTeX code
xlim(kgrid[1],kgrid[Nk])
ylim(kgrid[1],kgrid[Nk])
for i in 1:N
    l=ax.plot([kgrid[1],ks[i],ks[i]],[ks[i],ks[i],kgrid[1]],"k",linewidth=1,dashes=(8,4))
end
title("Next Period Capital Stock",fontsize=fst)

ax=subplot(3,3,5,facecolor=bg)
for sp in ("bottom","top","left","right")
      ax.spines[sp].set_color("none")
      ax.tick_params(axis="y", width=0, length=0)
      ax.tick_params(axis="x", width=0, length=0)        
  end
ax.grid(color="w",linestyle="-",alpha=0.5,linewidth=1.5,zorder=0)
ax.plot(kgrid,Gc,color="k",linewidth=1)                             # Plots k(t+1)=G(k(t))
xlabel(L"k_t",fontsize=12)                                         # put a label on x-axis the L forces LaTeX code
ylabel(L"c_t=G_c(k_t)",fontsize=12)                              # put a label on y-axis the L forces LaTeX code
title("Consumption",fontsize=fst)
xlim(kgrid[1],kgrid[Nk])

ax=subplot(3,3,6,facecolor=bg)
for sp in ("bottom","top","left","right")
      ax.spines[sp].set_color("none")
      ax.tick_params(axis="y", width=0, length=0)
      ax.tick_params(axis="x", width=0, length=0)        
  end
ax.grid(color="w",linestyle="-",alpha=0.5,linewidth=1.5,zorder=0)
ax.plot(kgrid,Gk.-(1-δ)*kgrid,color="k",linewidth=1)                             # Plots k(t+1)=G(k(t))
xlabel(L"k_t",fontsize=12)                                         # put a label on x-axis the L forces LaTeX code
ylabel(L"i_t=G_i(k_t)",fontsize=12)                              # put a label on y-axis the L forces LaTeX code
title("Investment",fontsize=fst)
xlim(kgrid[1],kgrid[Nk])

tight_layout()
savefig("vi_irrev.pdf",format="pdf",bbox_inches="tight",pad_inches=0.05)

**Now let's extend the previous method to another one**
It is important to learn the endogeneous grid method. Let's try to replicate the same results but using EGM. Let's code the functions step by step. 


In [ ]:
#From last to first steps. 1 derivative at different points of the grid. Here we assume that the function is differentiable, has no kinks etc. 

function deriv(v,k,i)

    λ= (k[i+1]-k[i])/(k[i]- k[i-1])
    dv1= ((v[i+1]-v[i]))/(k[i+1]-k[i])
    dv2= ((v[i]-v[i-1]))/(k[i]- k[i-1])
    dv= λ* dv1 + (1-λ)dv2

end 

deriv (generic function with 1 method)

In [ ]:
#function 2